In [1]:
# ================================================================================
# 담당자별 리드 전환 예측 특화 대시보드 - AI/ML 성능 개선 버전
# 플랫폼: Google Colab / Gradio 5.0+
# 목표: 고성능 담당자별 리드 전환율 예측 및 최적 배정 시스템
# ================================================================================

# 1. Google Colab 환경 설정 및 라이브러리 자동 설치
import subprocess
import sys
import os

def install_packages():
    """필요한 패키지 자동 설치"""
    packages = [
        'gradio>=5.0.0',
        'pandas',
        'numpy',
        'scikit-learn>=1.0.0',  # 버전 명시
        'lightgbm',
        'xgboost',
        'catboost',  # 추가: 더 강력한 그래디언트 부스팅
        'optuna>=3.0.0',  # 하이퍼파라미터 최적화 (버전 명시)
        'plotly',
        'seaborn',
        'matplotlib',
        'openpyxl',
        'japanize-matplotlib',
        'tensorflow',
        'imbalanced-learn>=0.9.0',  # 추가: 클래스 불균형 처리 (버전 명시)
        'feature-engine'  # 추가: 고급 피처 엔지니어링
    ]

    print("📦 필요한 패키지 설치 중...")

    # 먼저 pip 업그레이드
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
    except:
        pass

    # 기본 패키지 설치
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
            print(f"✅ {package} 설치 완료")
        except:
            print(f"⚠️ {package} 설치 실패, 계속 진행...")

    # 시계열 패키지는 별도로 처리 (오류가 자주 발생)
    print("\n📈 시계열 분석 패키지 설치 중...")

    # statsmodels 설치 시도
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'statsmodels>=0.13.0'])
        print("✅ statsmodels 설치 완료")
    except:
        print("⚠️ statsmodels 설치 실패 - ARIMA 모델을 사용할 수 없습니다")

    # pmdarima 설치 시도 (의존성이 복잡해서 자주 실패함)
    try:
        # 먼저 의존성 설치
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'cython', 'numpy', 'scipy'])
        # pmdarima 설치
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'pmdarima>=2.0.0'])
        print("✅ pmdarima 설치 완료")
    except:
        print("⚠️ pmdarima 설치 실패 - Auto ARIMA를 사용할 수 없습니다")

    # prophet 설치 시도
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'prophet>=1.1'])
        print("✅ prophet 설치 완료")
    except:
        print("⚠️ prophet 설치 실패 - Prophet 모델을 사용할 수 없습니다")

    # 한글 폰트 설치 (Colab용)
    try:
        print("\n🎨 한글 폰트 설치 중...")
        subprocess.run(['apt-get', 'update', '-qq'], check=True, capture_output=True)
        subprocess.run(['apt-get', 'install', '-qq', 'fonts-nanum*'], check=True, capture_output=True)
        subprocess.run(['fc-cache', '-fv'], capture_output=True)
    except:
        print("⚠️ 한글 폰트 설치 실패, 기본 폰트 사용")

    print("\n✅ 패키지 설치 완료!")
    print("💡 일부 시계열 패키지가 설치되지 않아도 기본 ML 모델은 정상 작동합니다.")

# 패키지 설치 실행
install_packages()

# 2. 라이브러리 임포트
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
try:
    import japanize_matplotlib
    japanize_matplotlib.japanize()
except:
    # 대체 폰트 설정
    plt.rcParams['font.family'] = ['DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, average_precision_score, mean_squared_error
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, VarianceThreshold
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime, timedelta
import io
import json
import base64
from typing import Dict, List, Tuple, Optional, Any

# 추가 라이브러리
try:
    import catboost as cb
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except:
    print("⚠️ CatBoost를 사용할 수 없습니다.")
    CATBOOST_AVAILABLE = False

try:
    import optuna
    OPTUNA_AVAILABLE = True
except:
    print("⚠️ Optuna를 사용할 수 없습니다.")
    OPTUNA_AVAILABLE = False

try:
    from imblearn.over_sampling import SMOTE, ADASYN
    from imblearn.under_sampling import RandomUnderSampler
    from imblearn.combine import SMOTEENN
    IMBALANCED_AVAILABLE = True
except:
    print("⚠️ Imbalanced-learn을 사용할 수 없습니다.")
    IMBALANCED_AVAILABLE = False

# 시계열 모델 라이브러리
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    TENSORFLOW_AVAILABLE = True
    print("✅ TensorFlow 로드 성공")
except Exception as e:
    print(f"⚠️ TensorFlow를 사용할 수 없습니다. LSTM 모델은 비활성화됩니다. (오류: {str(e)})")
    TENSORFLOW_AVAILABLE = False

try:
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.stattools import adfuller
    import pmdarima as pm
    STATSMODELS_AVAILABLE = True
    print("✅ Statsmodels/pmdarima 로드 성공")
except ImportError as e:
    print(f"⚠️ Statsmodels를 사용할 수 없습니다. ARIMA 모델은 비활성화됩니다.")
    print(f"   원인: {str(e)}")
    print("   해결방법: !pip install statsmodels pmdarima")
    STATSMODELS_AVAILABLE = False
except Exception as e:
    print(f"⚠️ Statsmodels 로드 중 예상치 못한 오류: {str(e)}")
    STATSMODELS_AVAILABLE = False

try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
    print("✅ Prophet 로드 성공")
except Exception as e:
    print(f"⚠️ Prophet을 사용할 수 없습니다. (오류: {str(e)})")
    PROPHET_AVAILABLE = False

# ================================================================================
# 3. 전역 변수 및 상태 관리
# ================================================================================

class AppState:
    """애플리케이션 상태 관리 클래스"""
    def __init__(self):
        self.data = None
        self.processed_data = None
        self.best_model = None
        self.best_model_name = None
        self.feature_names = None
        self.encoders = None
        self.model_results = {}
        self.manager_performance = None
        self.prediction_results = None
        self.feature_importance = None
        self.is_data_loaded = False
        self.is_preprocessed = False
        self.is_model_trained = False
        self.time_series_models = {}  # 시계열 모델 저장
        self.ensemble_model = None  # 앙상블 모델
        self.feature_selector = None  # 피처 선택기
        self.scaler = None  # 스케일러

app_state = AppState()

# ================================================================================
# 4. 샘플 데이터 생성 함수
# ================================================================================

def create_sample_data() -> pd.DataFrame:
    """tst_atalk.xlsx와 동일한 구조의 샘플 데이터 생성"""
    np.random.seed(42)
    n_samples = 500

    # 담당자 목록 (실제 데이터와 동일)
    managers = [
        '배제경(sv5203)', '예신해(sv0027)', '양은서(sv5200)',
        '김태연(sv0026)', '윤석한(sv5202)', '정지웅(sv0025)',
        '박온설아(sv0024)', None  # None은 미배정
    ]

    # 판매처 목록
    channels = ['G마켓', 'CJ몰', '11번가', 'SSG', '옥션', '네이버', '쿠팡']

    # 상태 목록
    statuses = ['승인완료', '1회 접촉', '2회 접촉', '3회 접촉', '접촉실패', '취소완료', '접수완료']
    status_weights = [0.25, 0.15, 0.20, 0.10, 0.10, 0.05, 0.15]

    # 데이터 생성
    data = {
        '공유': ['공용'] * n_samples,
        '이름': [f'고객_{np.random.randint(1000, 9999)}' for _ in range(n_samples)],
        '고객번호': [f'452036_{i}' for i in range(n_samples)],
        '성별': np.random.choice(['남자', '여자'], n_samples, p=[0.4, 0.6]),
        '유형': np.random.choice(['개인', '법인'], n_samples, p=[0.9, 0.1]),
        '생일구분': ['양력'] * n_samples,
        '혼인여부': np.random.choice(['미혼', '기혼'], n_samples, p=[0.4, 0.6]),
        'TEL': [f'10{np.random.randint(10000000, 99999999)}' for _ in range(n_samples)],
        '메모': ['LG정수기렌탈 상담 문의 ' * np.random.randint(1, 5) for _ in range(n_samples)],
        '상태': np.random.choice(statuses, n_samples, p=status_weights),
        '마케팅활용동의': np.random.choice(['동의', '미동의'], n_samples, p=[0.7, 0.3]),
        '이메일수신동의': np.random.choice(['동의', '미동의'], n_samples, p=[0.6, 0.4]),
        '문자수신동의': np.random.choice(['동의', '미동의'], n_samples, p=[0.65, 0.35]),
        '담당자': np.random.choice(managers, n_samples, p=[0.2, 0.17, 0.16, 0.15, 0.15, 0.13, 0.03, 0.01]),
        '등록일자': pd.date_range(start='2024-01-01', periods=n_samples, freq='H'),
        '판매처': np.random.choice(channels, n_samples),
        '등록자': np.random.choice(['양은서', '김민수', '이지은'], n_samples),
        '등록일시': pd.date_range(start='2024-01-01', periods=n_samples, freq='H')
    }

    df = pd.DataFrame(data)

    # 담당자별로 다른 승인율 적용
    for idx, row in df.iterrows():
        if pd.notna(row['담당자']):
            if '배제경' in row['담당자']:
                if np.random.random() < 0.35:
                    df.at[idx, '상태'] = '승인완료'
            elif '예신해' in row['담당자']:
                if np.random.random() < 0.32:
                    df.at[idx, '상태'] = '승인완료'
            elif '김태연' in row['담당자']:
                if np.random.random() < 0.30:
                    df.at[idx, '상태'] = '승인완료'

    return df

# ================================================================================
# 5. 고급 피처 엔지니어링 함수
# ================================================================================

def advanced_feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    """고급 피처 엔지니어링"""

    # 담당자별 과거 성과 통계
    if '담당자명' in data.columns:
        manager_stats = data.groupby('담당자명').agg({
            'is_approved': ['mean', 'sum', 'count'],
            '접촉횟수': ['mean', 'std'],
            '메모_길이': 'mean'
        }).round(4)

        manager_stats.columns = [
            '담당자_승인율', '담당자_총승인수', '담당자_총리드수',
            '담당자_평균접촉', '담당자_접촉편차', '담당자_평균메모'
        ]

        # 담당자 통계 병합
        data = data.merge(manager_stats, left_on='담당자명', right_index=True, how='left')

        # 담당자 경험 지표
        data['담당자_경험지수'] = data['담당자_총리드수'] / data['담당자_총리드수'].max()
        data['담당자_효율성'] = data['담당자_승인율'] / (data['담당자_평균접촉'] + 1)

    # 시간대별 패턴
    if '등록일자' in data.columns and pd.api.types.is_datetime64_any_dtype(data['등록일자']):
        # 시간대별 특성
        data['시간대'] = pd.cut(data['등록_시간'],
                               bins=[0, 6, 12, 18, 24],
                               labels=['새벽', '오전', '오후', '저녁'])

        # 주말 여부
        data['주말여부'] = (data['등록_요일'] >= 5).astype(int)

        # 월초/월말 구분
        data['월초'] = (data['등록_일'] <= 10).astype(int)
        data['월말'] = (data['등록_일'] >= 21).astype(int)

        # 분기
        data['분기'] = data['등록일자'].dt.quarter

    # 판매처별 특성
    if '판매처' in data.columns:
        channel_stats = data.groupby('판매처')['is_approved'].agg(['mean', 'count'])
        channel_stats.columns = ['판매처_평균승인율', '판매처_빈도']
        data = data.merge(channel_stats, left_on='판매처', right_index=True, how='left')

    # 고객 특성 조합
    if all(col in data.columns for col in ['성별', '혼인여부', '유형']):
        data['고객프로파일'] = data['성별'] + '_' + data['혼인여부'] + '_' + data['유형']

        # 프로파일별 승인율
        profile_stats = data.groupby('고객프로파일')['is_approved'].mean()
        data['프로파일_승인율'] = data['고객프로파일'].map(profile_stats)

    # 동의 점수
    consent_cols = ['마케팅활용동의_num', '이메일수신동의_num', '문자수신동의_num']
    if all(col in data.columns for col in consent_cols):
        data['동의점수'] = data[consent_cols].sum(axis=1)
        data['완전동의'] = (data['동의점수'] == 3).astype(int)

    # 메모 관련 고급 피처
    if '메모' in data.columns:
        # 특정 키워드 포함 여부
        keywords = ['정수기', '렌탈', '상담', '문의', '견적', '설치']
        for keyword in keywords:
            data[f'메모_{keyword}_포함'] = data['메모'].str.contains(keyword, na=False).astype(int)

        # 메모 품질 점수
        data['메모_품질점수'] = (
            data['메모_길이'] / data['메모_길이'].max() * 0.3 +
            data['메모_단어수'] / data['메모_단어수'].max() * 0.3 +
            data[[f'메모_{kw}_포함' for kw in keywords]].sum(axis=1) / len(keywords) * 0.4
        )

    # 상호작용 피처
    if '담당자_승인율' in data.columns and '판매처_평균승인율' in data.columns:
        data['담당자X판매처_시너지'] = data['담당자_승인율'] * data['판매처_평균승인율']

    if '접촉횟수' in data.columns and '메모_길이' in data.columns:
        data['접촉X메모_상호작용'] = data['접촉횟수'] * np.log1p(data['메모_길이'])

    # 이상치 플래그
    numerical_cols = data.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        if col not in ['is_approved', 'is_approved_binary']:
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            data[f'{col}_이상치'] = ((data[col] < lower) | (data[col] > upper)).astype(int)

    return data

# ================================================================================
# 6. 데이터 처리 함수 (개선된 버전)
# ================================================================================

def load_data(file) -> Tuple[pd.DataFrame, str]:
    """데이터 로드 및 초기 분석"""
    try:
        if file is None:
            # 샘플 데이터 사용
            df = create_sample_data()
            info_text = "📌 샘플 데이터를 사용합니다.\n"
        else:
            # 파일 읽기
            filename = file.name if hasattr(file, 'name') else 'uploaded_file'

            try:
                if filename.endswith('.csv'):
                    df = pd.read_csv(file, encoding='utf-8')
                elif filename.endswith(('.xlsx', '.xls')):
                    df = pd.read_excel(file)
                else:
                    return None, "❌ 지원되지 않는 파일 형식입니다. CSV 또는 Excel 파일을 업로드해주세요."
            except Exception as e:
                return None, f"❌ 파일 읽기 오류: {str(e)}"

        # 필수 컬럼 확인
        required_columns = ['상태', '담당자']
        missing_columns = [col for col in required_columns if col not in df.columns]

        if missing_columns:
            return None, f"❌ 필수 컬럼이 없습니다: {', '.join(missing_columns)}"

        # 데이터 기본 정보 분석
        total_rows = len(df)

        # 상태별 분포 분석
        status_dist = df['상태'].value_counts().to_dict()
        approval_count = df[df['상태'] == '승인완료'].shape[0]
        approval_rate = (approval_count / total_rows * 100) if total_rows > 0 else 0

        # 담당자별 분포 분석
        manager_dist = df['담당자'].value_counts().head(10).to_dict()

        # 판매처별 분포 분석
        channel_dist = {}
        if '판매처' in df.columns:
            channel_dist = df['판매처'].value_counts().head(10).to_dict()

        info_text = f"""✅ 데이터 로딩 성공!

📊 데이터 개요:
- 전체 리드 수: {total_rows:,}개
- 승인완료: {approval_count}건 ({approval_rate:.1f}%)
- 컬럼 수: {len(df.columns)}개

📈 상태별 분포:
{chr(10).join([f'- {status}: {count}건' for status, count in list(status_dist.items())[:5]])}

👥 상위 담당자 (리드 수):
{chr(10).join([f'- {manager}: {count}건' for manager, count in list(manager_dist.items())[:5]])}

🏪 주요 판매처:
{chr(10).join([f'- {channel}: {count}건' for channel, count in list(channel_dist.items())[:3]])}

✅ 데이터 로드 완료! 이제 [전처리 실행] 버튼을 클릭하세요.
"""

        app_state.data = df
        app_state.is_data_loaded = True
        return df, info_text

    except Exception as e:
        return None, f"❌ 데이터 로딩 중 오류 발생: {str(e)}"

def preprocess_for_prediction(df: pd.DataFrame = None) -> Tuple[pd.DataFrame, str]:
    """리드 전환 예측을 위한 고급 데이터 전처리"""
    try:
        if df is None:
            if app_state.data is None:
                return None, "❌ 먼저 데이터를 로드해주세요."
            df = app_state.data

        data = df.copy()

        # 타겟 변수 생성
        data['is_approved'] = (data['상태'] == '승인완료').astype(int)

        # 담당자 정보 정제
        data['담당자'] = data['담당자'].fillna('미배정')
        data['담당자명'] = data['담당자'].str.extract(r'([^(]+)')[0].str.strip()
        data['담당자명'] = data['담당자명'].fillna('미배정')

        # 날짜 처리
        if '등록일자' in data.columns:
            data['등록일자'] = pd.to_datetime(data['등록일자'], errors='coerce')
            data['등록_년'] = data['등록일자'].dt.year
            data['등록_월'] = data['등록일자'].dt.month
            data['등록_일'] = data['등록일자'].dt.day
            data['등록_요일'] = data['등록일자'].dt.dayofweek
            data['등록_시간'] = data['등록일자'].dt.hour
        else:
            # 기본값 설정
            data['등록_년'] = 2024
            data['등록_월'] = 1
            data['등록_일'] = 1
            data['등록_요일'] = 0
            data['등록_시간'] = 9

        # 접촉 횟수 추출
        data['접촉횟수'] = 0
        contact_pattern = r'(\d+)회 접촉'
        contact_matches = data['상태'].str.extract(contact_pattern)
        data.loc[contact_matches[0].notna(), '접촉횟수'] = contact_matches[0].fillna(0).astype(int)

        # 메모 길이 계산
        if '메모' in data.columns:
            data['메모_길이'] = data['메모'].fillna('').str.len()
            data['메모_단어수'] = data['메모'].fillna('').str.split().str.len()
        else:
            data['메모_길이'] = 50
            data['메모_단어수'] = 10

        # 동의 정보 수치화
        consent_cols = ['마케팅활용동의', '이메일수신동의', '문자수신동의']
        for col in consent_cols:
            if col in data.columns:
                data[f'{col}_num'] = (data[col] == '동의').astype(int)
            else:
                data[f'{col}_num'] = 1  # 기본값

        # 고급 피처 엔지니어링 적용
        data = advanced_feature_engineering(data)

        # 담당자별 성과 통계 계산
        manager_stats = data[data['담당자명'] != '미배정'].groupby('담당자명').agg({
            'is_approved': ['count', 'sum', 'mean'],
            '접촉횟수': 'mean'
        }).round(3)

        if len(manager_stats) > 0:
            manager_stats.columns = ['총_리드수', '승인완료수', '승인완료율', '평균_접촉횟수']
            manager_stats = manager_stats.reset_index()
            manager_stats = manager_stats.sort_values('승인완료율', ascending=False)
        else:
            manager_stats = pd.DataFrame()

        # 전처리 결과 요약
        feature_count = len([col for col in data.columns if col not in df.columns])

        result_text = f"""✅ 고급 데이터 전처리 완료!

📊 타겟 변수 분포:
- 승인완료: {data['is_approved'].sum():,}건 ({data['is_approved'].mean():.1%})
- 미승인: {(1-data['is_approved']).sum():,}건

🔧 피처 엔지니어링:
- 생성된 신규 피처: {feature_count}개
- 담당자별 성과 지표 추가
- 시간대별 패턴 추출
- 고객 프로파일 분석
- 상호작용 피처 생성

👥 담당자별 성과 (상위 5명):
{manager_stats.head().to_string(index=False) if len(manager_stats) > 0 else '담당자 데이터 없음'}

📈 주요 피처 생성:
- 담당자 경험 및 효율성 지표
- 시간대별 패턴 (주말, 시간대, 월초/월말)
- 판매처별 성과 지표
- 고객 프로파일별 승인율
- 메모 품질 점수
- 이상치 탐지 플래그

✅ 전처리 완료! 이제 다른 탭에서 분석을 진행하세요.
"""

        app_state.processed_data = data
        app_state.manager_performance = manager_stats
        app_state.is_preprocessed = True
        return data, result_text

    except Exception as e:
        return None, f"❌ 전처리 중 오류 발생: {str(e)}\n\n상세 오류:\n{type(e).__name__}: {str(e)}"

# ================================================================================
# 7. 담당자별 성과 분석 및 시각화 (기존 유지)
# ================================================================================

def create_manager_performance_dashboard() -> Tuple[List[go.Figure], str]:
    """담당자별 성과 대시보드 생성"""
    try:
        if not app_state.is_preprocessed or app_state.processed_data is None:
            return [], "❌ 먼저 데이터를 로드하고 전처리를 실행해주세요."

        data = app_state.processed_data
        plots = []

        # 1. 담당자별 승인완료율 및 처리량
        manager_stats = data[data['담당자명'] != '미배정'].groupby('담당자명').agg({
            'is_approved': ['count', 'sum', 'mean']
        }).round(3)

        if len(manager_stats) == 0:
            return [], "❌ 담당자 데이터가 없습니다."

        manager_stats.columns = ['총_리드수', '승인완료수', '승인완료율']
        manager_stats = manager_stats.reset_index()
        manager_stats = manager_stats[manager_stats['총_리드수'] >= 5]  # 5건 이상만 표시
        manager_stats = manager_stats.sort_values('승인완료율', ascending=False)

        # 그래프 1: 담당자별 성과 매트릭스
        fig1 = px.scatter(manager_stats,
                         x='총_리드수',
                         y='승인완료율',
                         size='승인완료수',
                         text='담당자명',
                         color='승인완료율',
                         title='담당자별 성과 매트릭스 (리드 처리량 vs 승인완료율)',
                         color_continuous_scale='RdYlGn')

        fig1.update_traces(textposition='top center', textfont_size=10)
        fig1.update_layout(
            height=500,
            xaxis_title='총 리드 처리수',
            yaxis_title='승인완료율',
            yaxis_tickformat='.0%'
        )

        # 평균선 추가
        avg_approval_rate = data['is_approved'].mean()
        fig1.add_hline(y=avg_approval_rate, line_dash="dash", line_color="gray",
                       annotation_text=f"전체 평균: {avg_approval_rate:.1%}")

        plots.append(fig1)

        # 그래프 2: 담당자별 승인완료율 순위
        fig2 = go.Figure()

        # 상위 10명
        top_managers = manager_stats.head(10)
        fig2.add_trace(go.Bar(
            x=top_managers['담당자명'],
            y=top_managers['승인완료율'],
            text=top_managers['승인완료율'].apply(lambda x: f'{x:.1%}'),
            textposition='outside',
            name='승인완료율',
            marker_color='lightgreen'
        ))

        fig2.update_layout(
            title='담당자별 승인완료율 TOP 10',
            xaxis_title='담당자',
            yaxis_title='승인완료율',
            yaxis_tickformat='.0%',
            height=400
        )

        plots.append(fig2)

        # 그래프 3: 담당자별 일일 처리 패턴
        if '등록일자' in data.columns and pd.api.types.is_datetime64_any_dtype(data['등록일자']):
            daily_stats = data.groupby(['담당자명', pd.Grouper(key='등록일자', freq='D')]).agg({
                'is_approved': ['count', 'mean']
            }).reset_index()
            daily_stats.columns = ['담당자명', '날짜', '처리건수', '승인완료율']

            # 상위 5명 담당자만 표시
            top_5_managers = manager_stats.head(5)['담당자명'].tolist()
            daily_top = daily_stats[daily_stats['담당자명'].isin(top_5_managers)]

            if len(daily_top) > 0:
                fig3 = px.line(daily_top,
                              x='날짜',
                              y='승인완료율',
                              color='담당자명',
                              title='상위 5명 담당자의 일별 승인완료율 추이',
                              markers=True)

                fig3.update_layout(
                    height=400,
                    yaxis_tickformat='.0%',
                    hovermode='x unified'
                )

                plots.append(fig3)

        # 그래프 4: 담당자별 접촉 효율성
        contact_stats = data[data['담당자명'] != '미배정'].groupby('담당자명').agg({
            '접촉횟수': 'mean',
            'is_approved': 'mean'
        }).reset_index()

        contact_stats = contact_stats[contact_stats['접촉횟수'] > 0]

        if len(contact_stats) > 0:
            contact_stats['효율성'] = contact_stats['is_approved'] / (contact_stats['접촉횟수'] + 1)  # +1 to avoid division by zero
            contact_stats = contact_stats.sort_values('효율성', ascending=False).head(15)

            fig4 = go.Figure()
            fig4.add_trace(go.Bar(
                x=contact_stats['담당자명'],
                y=contact_stats['효율성'],
                text=contact_stats['효율성'].apply(lambda x: f'{x:.3f}'),
                textposition='outside',
                name='접촉 효율성',
                marker_color='lightcoral'
            ))

            fig4.update_layout(
                title='담당자별 접촉 효율성 (승인율/평균접촉횟수)',
                xaxis_title='담당자',
                yaxis_title='효율성 지수',
                height=400
            )

            plots.append(fig4)

        success_message = "✅ 성과 분석 완료!"
        return plots, success_message

    except Exception as e:
        return [], f"❌ 성과 분석 중 오류 발생: {str(e)}"

# ================================================================================
# 8. 하이퍼파라미터 최적화
# ================================================================================

def optimize_hyperparameters(X_train, y_train, model_type='xgboost'):
    """Optuna를 사용한 하이퍼파라미터 최적화 (오류 처리 강화)"""
    if not OPTUNA_AVAILABLE:
        return None

    try:
        def objective(trial):
            try:
                if model_type == 'xgboost':
                    params = {
                        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
                        'max_depth': trial.suggest_int('max_depth', 3, 8),
                        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
                        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
                        'gamma': trial.suggest_float('gamma', 0, 3),
                        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
                        'random_state': 42,
                        'use_label_encoder': False,
                        'eval_metric': 'logloss'
                    }
                    model = xgb.XGBClassifier(**params)

                elif model_type == 'lightgbm':
                    params = {
                        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
                        'max_depth': trial.suggest_int('max_depth', 3, 8),
                        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
                        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
                        'random_state': 42,
                        'verbose': -1
                    }
                    model = lgb.LGBMClassifier(**params)

                elif model_type == 'catboost':
                    if not CATBOOST_AVAILABLE:
                        return 0
                    params = {
                        'iterations': trial.suggest_int('iterations', 50, 150),
                        'depth': trial.suggest_int('depth', 3, 8),
                        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 5),
                        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 0.5),
                        'random_strength': trial.suggest_float('random_strength', 0, 0.5),
                        'random_state': 42,
                        'verbose': False
                    }
                    model = CatBoostClassifier(**params)

                # 간단한 교차 검증 (시계열 분할 대신 일반 분할 사용)
                if len(X_train) < 100:
                    # 데이터가 적은 경우 단순 학습/검증 분할
                    X_tr, X_val, y_tr, y_val = train_test_split(
                        X_train, y_train, test_size=0.2, random_state=42
                    )
                    model.fit(X_tr, y_tr)
                    y_pred_proba = model.predict_proba(X_val)[:, 1]
                    score = roc_auc_score(y_val, y_pred_proba)
                else:
                    # 충분한 데이터가 있는 경우 교차 검증
                    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
                    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
                    score = scores.mean()

                return score

            except Exception as e:
                print(f"Trial 실패: {str(e)}")
                return 0.0

        # 최적화 수행 (시도 횟수 줄임)
        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=10, show_progress_bar=False, catch=(Exception,))

        # 완료된 trial이 있는지 확인
        if len(study.trials) == 0 or study.best_trial is None:
            print(f"{model_type} 최적화 실패 - 기본 파라미터 사용")
            return None

        return study.best_params

    except Exception as e:
        print(f"하이퍼파라미터 최적화 중 오류: {str(e)}")
        return None

# ================================================================================
# 9. 개선된 머신러닝 모델 학습
# ================================================================================

def handle_class_imbalance(X_train, y_train):
    """클래스 불균형 처리 (안전한 버전)"""
    if not IMBALANCED_AVAILABLE:
        return X_train, y_train

    try:
        # 클래스 비율 확인
        unique, counts = np.unique(y_train, return_counts=True)
        min_class_count = min(counts)

        # 소수 클래스가 너무 적으면 SMOTE 사용 불가
        if min_class_count < 6:
            print("소수 클래스 샘플이 너무 적어 불균형 처리를 건너뜁니다.")
            return X_train, y_train

        # SMOTE만 사용 (SMOTEENN은 때때로 불안정함)
        smote = SMOTE(random_state=42, k_neighbors=min(5, min_class_count-1))
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

        print(f"클래스 불균형 처리 완료: {len(X_train)} → {len(X_resampled)}")
        return X_resampled, y_resampled

    except Exception as e:
        print(f"클래스 불균형 처리 중 오류: {str(e)}")
        # 실패 시 원본 반환
        return X_train, y_train

def select_best_features(X_train, y_train, X_test, feature_names, k=20):
    """최적 피처 선택 (안전한 버전)"""
    try:
        # 피처 수가 k보다 적으면 모든 피처 사용
        if X_train.shape[1] <= k:
            return X_train, X_test, feature_names, None

        # 분산이 0인 피처 제거
        var_selector = VarianceThreshold(threshold=0.01)
        X_train_var = var_selector.fit_transform(X_train)
        X_test_var = var_selector.transform(X_test)
        selected_features_var = [feature_names[i] for i in var_selector.get_support(indices=True)]

        # 피처가 너무 많이 제거되었다면 원본 사용
        if len(selected_features_var) < 5:
            return X_train, X_test, feature_names, None

        # 상호 정보량 기반 피처 선택
        selector = SelectKBest(score_func=mutual_info_classif, k=min(k, len(selected_features_var)))
        selector.fit(X_train_var, y_train)

        # 선택된 피처 인덱스
        selected_indices = selector.get_support(indices=True)
        final_features = [selected_features_var[i] for i in selected_indices]

        # 변환
        X_train_selected = selector.transform(X_train_var)
        X_test_selected = selector.transform(X_test_var)

        print(f"피처 선택 완료: {X_train.shape[1]} → {X_train_selected.shape[1]}")

        return X_train_selected, X_test_selected, final_features, selector

    except Exception as e:
        print(f"피처 선택 중 오류: {str(e)}")
        return X_train, X_test, feature_names, None

def build_advanced_lstm_model(input_shape: Tuple[int, int]) -> keras.Model:
    """개선된 LSTM 모델 구축"""
    if not TENSORFLOW_AVAILABLE:
        return None

    model = keras.Sequential([
        layers.LSTM(128, return_sequences=True, input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.LSTM(64, return_sequences=True),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.LSTM(32, return_sequences=False),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'AUC', keras.metrics.Precision(), keras.metrics.Recall()]
    )

    return model

def train_prophet_for_manager(manager_data: pd.DataFrame) -> Any:
    """Prophet 모델 학습"""
    if not PROPHET_AVAILABLE:
        return None

    try:
        # Prophet 형식으로 데이터 준비
        prophet_data = manager_data[['날짜', '승인율']].rename(
            columns={'날짜': 'ds', '승인율': 'y'}
        )

        # 모델 학습
        model = Prophet(
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10,
            holidays_prior_scale=10,
            daily_seasonality=False,
            weekly_seasonality=True,
            yearly_seasonality=False
        )

        model.fit(prophet_data)
        return model

    except Exception as e:
        print(f"Prophet 학습 오류: {str(e)}")
        return None

def train_prediction_model_with_data(filtered_data: pd.DataFrame = None) -> Tuple[Dict, str]:
    """개선된 리드 전환 예측 모델 학습"""
    try:
        if filtered_data is None:
            filtered_data = app_state.processed_data

        if filtered_data is None or len(filtered_data) == 0:
            return {}, "❌ 학습할 데이터가 없습니다."

        data = filtered_data

        # 타겟 변수 확인
        if 'is_approved' not in data.columns:
            return {}, "❌ 타겟 변수(is_approved)가 없습니다."

        # 피처 준비
        feature_cols = []

        # 수치형 피처 (확장된 버전)
        numerical_features = [
            '접촉횟수', '메모_길이', '메모_단어수',
            '등록_월', '등록_일', '등록_요일', '등록_시간',
            '담당자_승인율', '담당자_총승인수', '담당자_총리드수',
            '담당자_평균접촉', '담당자_경험지수', '담당자_효율성',
            '판매처_평균승인율', '판매처_빈도', '프로파일_승인율',
            '동의점수', '완전동의', '메모_품질점수',
            '담당자X판매처_시너지', '접촉X메모_상호작용'
        ]

        # 범주형 피처
        categorical_features = ['담당자명', '판매처', '성별', '유형', '시간대', '고객프로파일']

        # 동의 관련 피처
        consent_features = ['마케팅활용동의_num', '이메일수신동의_num', '문자수신동의_num']

        # 키워드 피처
        keyword_features = [col for col in data.columns if col.startswith('메모_') and col.endswith('_포함')]

        # 이상치 피처
        outlier_features = [col for col in data.columns if col.endswith('_이상치')]

        # 사용 가능한 피처만 선택
        all_features = numerical_features + consent_features + keyword_features + outlier_features
        for col in all_features:
            if col in data.columns:
                feature_cols.append(col)

        if len(feature_cols) == 0:
            return {}, "❌ 사용 가능한 피처가 없습니다."

        # 범주형 변수 인코딩
        encoded_data = data.copy()
        encoders = {}

        for col in categorical_features:
            if col in data.columns:
                try:
                    encoded_data[col] = encoded_data[col].fillna('Unknown')
                    le = LabelEncoder()
                    encoded_data[f'{col}_encoded'] = le.fit_transform(encoded_data[col].astype(str))
                    encoders[col] = le
                    feature_cols.append(f'{col}_encoded')
                except Exception as e:
                    print(f"⚠️ {col} 인코딩 실패: {str(e)}")

        # 훈련 데이터 준비
        X = encoded_data[feature_cols].fillna(0)
        y = encoded_data['is_approved']

        # 데이터 검증
        if len(X) < 50:
            return {}, f"❌ 학습 데이터가 너무 적습니다. (현재: {len(X)}건, 최소: 50건)"

        # 스케일링
        scaler = RobustScaler()  # 이상치에 강한 스케일러
        X_scaled = scaler.fit_transform(X)
        app_state.scaler = scaler

        # 데이터 분할
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X_scaled, y, test_size=0.2, random_state=42, stratify=y
            )
        except:
            X_train, X_test, y_train, y_test = train_test_split(
                X_scaled, y, test_size=0.2, random_state=42
            )

        # 클래스 불균형 처리
        X_train_balanced, y_train_balanced = handle_class_imbalance(X_train, y_train)

        # 피처 선택
        X_train_selected, X_test_selected, selected_features, feature_selector = select_best_features(
            X_train_balanced, y_train_balanced, X_test, feature_cols
        )
        app_state.feature_selector = feature_selector

        # 모델 정의
        models = {
            'Random Forest': RandomForestClassifier(
                n_estimators=100, random_state=42, max_depth=10,
                min_samples_split=5, min_samples_leaf=2,
                class_weight='balanced'
            ),
            'Gradient Boosting': GradientBoostingClassifier(
                n_estimators=100, random_state=42, max_depth=5,
                learning_rate=0.1, subsample=0.8
            )
        }

        # 하이퍼파라미터 최적화 시도 (선택적)
        optimization_attempted = False
        if OPTUNA_AVAILABLE and len(X_train_selected) >= 100:  # 충분한 데이터가 있을 때만
            try:
                print("하이퍼파라미터 최적화 시도 중...")

                # XGBoost 최적화
                xgb_params = optimize_hyperparameters(X_train_selected, y_train_balanced, 'xgboost')
                if xgb_params:
                    models['XGBoost (Optimized)'] = xgb.XGBClassifier(**xgb_params)
                    optimization_attempted = True

                # LightGBM 최적화
                lgb_params = optimize_hyperparameters(X_train_selected, y_train_balanced, 'lightgbm')
                if lgb_params:
                    models['LightGBM (Optimized)'] = lgb.LGBMClassifier(**lgb_params)
                    optimization_attempted = True

                # CatBoost 최적화
                if CATBOOST_AVAILABLE:
                    cat_params = optimize_hyperparameters(X_train_selected, y_train_balanced, 'catboost')
                    if cat_params:
                        models['CatBoost (Optimized)'] = CatBoostClassifier(**cat_params)
                        optimization_attempted = True

            except Exception as e:
                print(f"최적화 중 오류 발생: {str(e)}")
                optimization_attempted = False

        # 기본 모델 추가 (최적화 실패 시 또는 항상 포함)
        if not optimization_attempted or True:  # 항상 기본 모델도 포함
            models['XGBoost'] = xgb.XGBClassifier(
                n_estimators=100, random_state=42, max_depth=6,
                learning_rate=0.1, use_label_encoder=False, eval_metric='logloss'
            )
            models['LightGBM'] = lgb.LGBMClassifier(
                n_estimators=100, random_state=42, max_depth=6,
                learning_rate=0.1, verbose=-1
            )
            if CATBOOST_AVAILABLE:
                models['CatBoost'] = CatBoostClassifier(
                    iterations=100, random_state=42, depth=6,
                    learning_rate=0.1, verbose=False
                )

        # 시계열 모델 추가 (선택적 - 사용 가능한 모델만)
        time_series_models = {}
        time_series_attempted = 0

        # 시계열 데이터 준비
        if '등록일자' in data.columns:
            ts_data = prepare_time_series_data(data)

            if len(ts_data) > 0:
                # 담당자별 시계열 모델 학습
                for manager in ts_data['담당자'].unique()[:5]:  # 상위 5명만
                    manager_ts = ts_data[ts_data['담당자'] == manager]

                    # LSTM 모델 (TensorFlow 필요)
                    if TENSORFLOW_AVAILABLE and len(manager_ts) >= 30:
                        try:
                            lstm_model, lstm_scaler = train_lstm_for_manager(manager_ts)
                            if lstm_model:
                                time_series_models[f'LSTM_{manager}'] = {
                                    'model': lstm_model,
                                    'scaler': lstm_scaler,
                                    'type': 'lstm'
                                }
                                time_series_attempted += 1
                        except Exception as e:
                            print(f"LSTM 모델 학습 실패 ({manager}): {str(e)}")

                    # ARIMA 모델 (statsmodels 필요)
                    if STATSMODELS_AVAILABLE and len(manager_ts) >= 30:
                        try:
                            arima_model = train_arima_for_manager(manager_ts)
                            if arima_model:
                                time_series_models[f'ARIMA_{manager}'] = {
                                    'model': arima_model,
                                    'type': 'arima'
                                }
                                time_series_attempted += 1
                        except Exception as e:
                            print(f"ARIMA 모델 학습 실패 ({manager}): {str(e)}")

                    # Prophet 모델 (prophet 필요)
                    if PROPHET_AVAILABLE and len(manager_ts) >= 30:
                        try:
                            prophet_model = train_prophet_for_manager(manager_ts)
                            if prophet_model:
                                time_series_models[f'Prophet_{manager}'] = {
                                    'model': prophet_model,
                                    'type': 'prophet'
                                }
                                time_series_attempted += 1
                        except Exception as e:
                            print(f"Prophet 모델 학습 실패 ({manager}): {str(e)}")

        results = {}
        best_score = -1
        best_model = None
        best_model_name = None
        ensemble_models = []

        result_text = f"🤖 고급 모델 학습 결과 (학습 데이터: {len(X)}건):\n" + "="*50 + "\n\n"

        # 모델 학습 및 평가
        result_text += "📊 머신러닝 모델 성능:\n"
        for name, model in models.items():
            try:
                model.fit(X_train_selected, y_train_balanced)
                y_pred = model.predict(X_test_selected)
                y_pred_proba = model.predict_proba(X_test_selected)[:, 1]

                # 다양한 메트릭 계산
                roc_auc = roc_auc_score(y_test, y_pred_proba)
                f1 = f1_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)

                results[name] = {
                    'model': model,
                    'roc_auc': roc_auc,
                    'f1_score': f1,
                    'precision': precision,
                    'recall': recall,
                    'predictions': y_pred_proba,
                    'y_test': y_test,
                    'type': 'ml'
                }

                result_text += f"  - {name}:\n"
                result_text += f"    • ROC-AUC: {roc_auc:.4f}\n"
                result_text += f"    • F1-Score: {f1:.4f}\n"
                result_text += f"    • Precision: {precision:.4f}\n"
                result_text += f"    • Recall: {recall:.4f}\n"

                if roc_auc > best_score:
                    best_score = roc_auc
                    best_model = model
                    best_model_name = name

                # 앙상블에 포함할 모델 선택 (ROC-AUC > 0.6)
                if roc_auc > 0.6:
                    ensemble_models.append((name, model))

            except Exception as e:
                result_text += f"  ❌ {name} 학습 실패: {str(e)}\n"

        # 앙상블 모델 생성 (3개 이상의 모델이 성공한 경우)
        if len(ensemble_models) >= 2:  # 기준을 3개에서 2개로 낮춤
            try:
                voting_clf = VotingClassifier(
                    estimators=ensemble_models,
                    voting='soft'
                )
                voting_clf.fit(X_train_selected, y_train_balanced)
                y_pred_ensemble = voting_clf.predict_proba(X_test_selected)[:, 1]
                ensemble_auc = roc_auc_score(y_test, y_pred_ensemble)

                results['Ensemble'] = {
                    'model': voting_clf,
                    'roc_auc': ensemble_auc,
                    'predictions': y_pred_ensemble,
                    'y_test': y_test,
                    'type': 'ensemble'
                }

                result_text += f"\n🎯 앙상블 모델 (상위 {len(ensemble_models)}개 결합):\n"
                result_text += f"  - ROC-AUC: {ensemble_auc:.4f}\n"

                if ensemble_auc > best_score:
                    best_score = ensemble_auc
                    best_model = voting_clf
                    best_model_name = 'Ensemble'
                    app_state.ensemble_model = voting_clf

            except Exception as e:
                result_text += f"\n⚠️ 앙상블 생성 실패: {str(e)}\n"

        # 시계열 모델 결과 추가
        if time_series_models:
            result_text += "\n📈 시계열 전문 모델:\n"
            for name, model_info in time_series_models.items():
                result_text += f"  - {name}: 학습 완료\n"
                results[name] = model_info

        if best_model is None:
            return {}, "❌ 모든 모델 학습에 실패했습니다."

        result_text += f"\n🏆 최고 성능 모델: {best_model_name} (ROC-AUC: {best_score:.4f})\n"

        # 피처 중요도
        if hasattr(best_model, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature': selected_features,
                'importance': best_model.feature_importances_
            }).sort_values('importance', ascending=False).head(10)

            result_text += "\n📊 상위 10개 중요 피처:\n"
            for _, row in feature_importance.iterrows():
                result_text += f"  - {row['feature']}: {row['importance']:.4f}\n"

        # 시계열 모델 사용 가능 여부
        if time_series_models:
            result_text += f"\n🔮 시계열 모델 {len(time_series_models)}개 추가 학습 완료!"
            result_text += "\n   (사용 가능: "
            model_types = set([info['type'] for info in time_series_models.values()])
            result_text += ", ".join([t.upper() for t in model_types]) + ")"
        else:
            result_text += "\n📊 시계열 모델은 사용할 수 없지만, ML 모델로 충분한 예측이 가능합니다."

        result_text += "\n\n✅ 고급 모델 학습 완료!"

        # 상태 저장
        app_state.best_model = best_model
        app_state.best_model_name = best_model_name
        app_state.feature_names = selected_features
        app_state.encoders = encoders
        app_state.model_results = results
        app_state.time_series_models = time_series_models if time_series_models else {}
        app_state.is_model_trained = True

        # 담당자별 성과 재계산 (필터링된 데이터 기준)
        manager_stats = data[data['담당자명'] != '미배정'].groupby('담당자명').agg({
            'is_approved': ['count', 'sum', 'mean'],
            '접촉횟수': 'mean'
        }).round(3)

        if len(manager_stats) > 0:
            manager_stats.columns = ['총_리드수', '승인완료수', '승인완료율', '평균_접촉횟수']
            manager_stats = manager_stats.reset_index()
            app_state.manager_performance = manager_stats

        return results, result_text

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        error_message = f"❌ 모델 학습 중 오류 발생:\n{str(e)}\n\n"

        # 더 자세한 오류 정보 제공
        if "No trials are completed" in str(e):
            error_message += "💡 하이퍼파라미터 최적화에 실패했습니다. 기본 모델로 재시도 중...\n"

            # 기본 모델로 재시도
            try:
                # 간단한 모델로 재시도
                simple_model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=5)
                simple_model.fit(X_train[:, :min(10, X_train.shape[1])], y_train)  # 피처 수 제한

                app_state.best_model = simple_model
                app_state.best_model_name = "Random Forest (Basic)"
                app_state.is_model_trained = True

                return {"Random Forest (Basic)": {'model': simple_model}}, "✅ 기본 모델로 학습 완료!"

            except Exception as e2:
                error_message += f"\n기본 모델 학습도 실패: {str(e2)}"

        return {}, error_message

# ================================================================================
# 10. 나머지 함수들 (기존 코드 재사용)
# ================================================================================

# 기존 코드의 나머지 함수들을 그대로 포함
# - prepare_time_series_data
# - build_lstm_model (advanced 버전으로 대체됨)
# - train_lstm_for_manager
# - train_arima_for_manager
# - filter_data_by_date
# - create_time_series_prediction
# - train_prediction_model
# - predict_manager_performance
# - optimize_lead_assignment
# - create_gradio_interface

def prepare_time_series_data(data: pd.DataFrame) -> pd.DataFrame:
    """시계열 모델을 위한 데이터 준비"""
    try:
        if '등록일자' not in data.columns:
            return pd.DataFrame()

        # 일별 승인율 집계
        daily_stats = data.groupby([pd.Grouper(key='등록일자', freq='D'), '담당자명']).agg({
            'is_approved': ['count', 'sum', 'mean']
        }).reset_index()

        daily_stats.columns = ['날짜', '담당자', '리드수', '승인수', '승인율']
        daily_stats = daily_stats[daily_stats['리드수'] > 0]  # 리드가 있는 날만

        # 시계열 피처 추가
        daily_stats['요일'] = daily_stats['날짜'].dt.dayofweek
        daily_stats['월'] = daily_stats['날짜'].dt.month
        daily_stats['분기'] = daily_stats['날짜'].dt.quarter

        return daily_stats

    except Exception as e:
        print(f"시계열 데이터 준비 오류: {str(e)}")
        return pd.DataFrame()

def train_lstm_for_manager(manager_data: pd.DataFrame, sequence_length: int = 7) -> Tuple[keras.Model, StandardScaler]:
    """담당자별 LSTM 모델 학습 (개선된 버전)"""
    if not TENSORFLOW_AVAILABLE:
        return None, None

    try:
        # 데이터 준비
        data = manager_data.sort_values('날짜')

        # 특성과 타겟 분리
        features = ['리드수', '승인수', '요일', '월']
        X = data[features].values
        y = data['승인율'].values

        # 정규화
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # 시퀀스 생성
        X_seq, y_seq = [], []
        for i in range(len(X_scaled) - sequence_length):
            X_seq.append(X_scaled[i:i+sequence_length])
            y_seq.append(y[i+sequence_length])

        X_seq = np.array(X_seq)
        y_seq = np.array(y_seq)

        if len(X_seq) < 10:  # 데이터가 너무 적으면 학습 불가
            return None, None

        # 모델 구축 및 학습
        model = build_advanced_lstm_model((sequence_length, len(features)))

        # 콜백 설정
        early_stop = EarlyStopping(
            monitor='loss',
            patience=20,
            restore_best_weights=True
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='loss',
            factor=0.5,
            patience=10,
            min_lr=0.0001
        )

        # 학습
        model.fit(
            X_seq, y_seq,
            epochs=100,
            batch_size=16,
            verbose=0,
            callbacks=[early_stop, reduce_lr],
            validation_split=0.2
        )

        return model, scaler

    except Exception as e:
        print(f"LSTM 학습 오류: {str(e)}")
        return None, None

def train_arima_for_manager(manager_data: pd.DataFrame) -> Any:
    """담당자별 ARIMA 모델 학습"""
    if not STATSMODELS_AVAILABLE:
        return None

    try:
        # 시계열 데이터 준비
        ts_data = manager_data.set_index('날짜')['승인율']
        ts_data = ts_data.asfreq('D').ffill()  # 일별 주기로 변환

        if len(ts_data) < 30:  # 데이터가 너무 적으면 학습 불가
            return None

        # Auto ARIMA로 최적 파라미터 찾기
        try:
            model = pm.auto_arima(
                ts_data,
                seasonal=True,
                m=7,  # 주간 계절성
                stepwise=True,
                suppress_warnings=True,
                error_action='ignore',
                max_p=3,
                max_q=3,
                max_order=5,
                trace=False,  # 진행 상황 출력 안함
                n_jobs=1  # 단일 스레드 사용
            )
        except:
            # Auto ARIMA 실패 시 단순 ARIMA 시도
            from statsmodels.tsa.arima.model import ARIMA
            model = ARIMA(ts_data, order=(1,1,1))
            model = model.fit()

        return model

    except Exception as e:
        print(f"ARIMA 학습 오류 ({manager_data['담당자'].iloc[0] if len(manager_data) > 0 else '알 수 없음'}): {str(e)}")
        return None

def filter_data_by_date(start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.DataFrame:
    """지정된 날짜 범위의 데이터만 필터링"""
    try:
        if app_state.processed_data is None:
            return None

        data = app_state.processed_data.copy()

        # 날짜 컬럼이 있는 경우만 필터링
        if '등록일자' in data.columns:
            # 날짜 형식 확인 및 변환
            if not pd.api.types.is_datetime64_any_dtype(data['등록일자']):
                data['등록일자'] = pd.to_datetime(data['등록일자'], errors='coerce')

            # 날짜 범위로 필터링
            mask = (data['등록일자'] >= start_date) & (data['등록일자'] <= end_date)
            filtered_data = data[mask]

            print(f"날짜 필터링: {start_date.date()} ~ {end_date.date()}")
            print(f"필터링 전: {len(data)}건 → 필터링 후: {len(filtered_data)}건")

            return filtered_data
        else:
            # 날짜 컬럼이 없으면 전체 데이터 반환
            return data

    except Exception as e:
        print(f"날짜 필터링 오류: {str(e)}")
        return None

def create_time_series_prediction(start_date: pd.Timestamp, days: int) -> go.Figure:
    """시계열 예측 그래프 생성 (개선된 버전)"""
    try:
        if not app_state.is_model_trained or app_state.manager_performance is None:
            return None

        # 예측 날짜 범위 생성
        date_range = pd.date_range(start=start_date, periods=days, freq='D')

        # 담당자별 예측 데이터 생성
        predictions_by_date = []

        # 각 담당자의 일일 평균 리드 수 계산
        manager_daily_leads = {}
        for _, manager_row in app_state.manager_performance.head(5).iterrows():
            manager = manager_row['담당자명']
            daily_avg = manager_row['총_리드수'] / 30
            manager_daily_leads[manager] = daily_avg

        # 시계열 모델 사용 가능 여부 확인
        use_time_series = len(app_state.time_series_models) > 0

        for date in date_range:
            for _, manager_row in app_state.manager_performance.head(5).iterrows():
                manager = manager_row['담당자명']

                # 시계열 모델 우선 사용
                pred_proba = None
                model_used = "ML"
                confidence = 0.8  # 기본 신뢰도

                # Prophet 모델 확인 (최우선)
                prophet_key = f'Prophet_{manager}'
                if use_time_series and prophet_key in app_state.time_series_models:
                    try:
                        prophet_model = app_state.time_series_models[prophet_key]['model']
                        future = prophet_model.make_future_dataframe(periods=1)
                        forecast = prophet_model.predict(future)
                        pred_proba = forecast['yhat'].iloc[-1]
                        pred_proba = max(0, min(1, pred_proba))
                        model_used = "Prophet"
                        confidence = 0.9
                    except:
                        pass

                # LSTM 모델 확인
                lstm_key = f'LSTM_{manager}'
                if pred_proba is None and use_time_series and lstm_key in app_state.time_series_models:
                    try:
                        lstm_info = app_state.time_series_models[lstm_key]
                        lstm_model = lstm_info['model']
                        lstm_scaler = lstm_info['scaler']

                        # 최근 7일 데이터 준비 (시뮬레이션)
                        recent_features = np.array([
                            [manager_daily_leads[manager],
                             manager_daily_leads[manager] * manager_row['승인완료율'],
                             date.dayofweek,
                             date.month]
                        ])

                        recent_scaled = lstm_scaler.transform(recent_features)
                        sequence = np.tile(recent_scaled, (7, 1)).reshape(1, 7, -1)

                        pred_proba = lstm_model.predict(sequence)[0, 0]
                        model_used = "LSTM"
                        confidence = 0.85
                    except:
                        pass

                # ARIMA 모델 확인
                arima_key = f'ARIMA_{manager}'
                if pred_proba is None and use_time_series and arima_key in app_state.time_series_models:
                    try:
                        arima_model = app_state.time_series_models[arima_key]['model']
                        # ARIMA 예측 (1일 앞)
                        forecast = arima_model.forecast(steps=1)
                        pred_proba = forecast[0]
                        pred_proba = max(0, min(1, pred_proba))  # 0-1 범위로 제한
                        model_used = "ARIMA"
                        confidence = 0.8
                    except:
                        pass

                # 기본 ML 모델 사용
                if pred_proba is None:
                    # 더 많은 피처 사용
                    sample_data = app_state.processed_data[
                        app_state.processed_data['담당자명'] == manager
                    ].mean(numeric_only=True).to_dict()

                    # 날짜 관련 피처 업데이트
                    sample_data.update({
                        '등록_월': date.month,
                        '등록_일': date.day,
                        '등록_요일': date.dayofweek,
                        '등록_시간': 14,
                        '담당자명': manager
                    })

                    sample_df = pd.DataFrame([sample_data])

                    # 인코딩
                    for col, encoder in app_state.encoders.items():
                        if col in sample_df.columns:
                            try:
                                sample_df[f'{col}_encoded'] = encoder.transform(sample_df[col])
                            except:
                                sample_df[f'{col}_encoded'] = 0

                    # 피처 선택 및 스케일링
                    try:
                        X_pred = sample_df[app_state.feature_names].fillna(0)
                        if app_state.scaler:
                            X_pred = app_state.scaler.transform(X_pred)
                        if app_state.feature_selector:
                            X_pred = app_state.feature_selector.transform(X_pred)

                        pred_proba = app_state.best_model.predict_proba(X_pred)[0, 1]
                        confidence = 0.7
                    except:
                        pred_proba = manager_row['승인완료율']
                        confidence = 0.6

                # 예상 승인수 계산
                daily_leads = manager_daily_leads[manager]
                expected_approvals = daily_leads * pred_proba

                # 신뢰 구간 계산
                std_dev = expected_approvals * 0.1 * (1 - confidence)
                lower_bound = max(0, expected_approvals - 1.96 * std_dev)
                upper_bound = expected_approvals + 1.96 * std_dev

                predictions_by_date.append({
                    '날짜': date,
                    '담당자': manager,
                    '예상_승인수': expected_approvals,
                    '예상_승인율': pred_proba,
                    '예상_리드수': daily_leads,
                    '모델': model_used,
                    '신뢰도': confidence,
                    '하한': lower_bound,
                    '상한': upper_bound
                })

        # DataFrame 변환
        pred_df = pd.DataFrame(predictions_by_date)

        # 시계열 그래프 생성
        fig = make_subplots(
            rows=4, cols=1,
            subplot_titles=(
                f'{days}일간 담당자별 예상 승인수 (95% 신뢰구간)',
                '일별 전체 예상 승인수',
                '모델별 예측 기여도',
                '예측 신뢰도 분포'
            ),
            row_heights=[0.4, 0.25, 0.15, 0.2],
            vertical_spacing=0.12
        )

        # 1. 담당자별 예상 승인수 추이 (신뢰구간 포함)
        for manager in pred_df['담당자'].unique():
            manager_data = pred_df[pred_df['담당자'] == manager]

            # 주 추세선
            fig.add_trace(
                go.Scatter(
                    x=manager_data['날짜'],
                    y=manager_data['예상_승인수'],
                    name=f"{manager}",
                    mode='lines+markers',
                    hovertemplate='%{y:.1f}건<br>승인율: %{customdata:.1%}',
                    customdata=manager_data['예상_승인율']
                ),
                row=1, col=1
            )

            # 신뢰구간
            fig.add_trace(
                go.Scatter(
                    x=manager_data['날짜'].tolist() + manager_data['날짜'].tolist()[::-1],
                    y=manager_data['상한'].tolist() + manager_data['하한'].tolist()[::-1],
                    fill='toself',
                    fillcolor=f'rgba(0,100,200,0.1)',
                    line=dict(color='rgba(255,255,255,0)'),
                    hoverinfo="skip",
                    showlegend=False,
                    name=f'{manager}_신뢰구간'
                ),
                row=1, col=1
            )

        # 2. 전체 예상 승인수
        daily_total = pred_df.groupby('날짜').agg({
            '예상_승인수': 'sum',
            '하한': 'sum',
            '상한': 'sum'
        }).reset_index()

        fig.add_trace(
            go.Bar(
                x=daily_total['날짜'],
                y=daily_total['예상_승인수'],
                name='전체 승인수',
                marker_color='lightblue',
                hovertemplate='%{y:.0f}건',
                error_y=dict(
                    type='data',
                    symmetric=False,
                    array=daily_total['상한'] - daily_total['예상_승인수'],
                    arrayminus=daily_total['예상_승인수'] - daily_total['하한']
                ),
                showlegend=False
            ),
            row=2, col=1
        )

        # 3. 모델별 사용 비율
        model_counts = pred_df['모델'].value_counts()

        fig.add_trace(
            go.Bar(
                x=model_counts.index,
                y=model_counts.values,
                name='모델 사용 횟수',
                marker_color=['darkgreen' if 'Prophet' in x else
                             'green' if 'LSTM' in x else
                             'orange' if 'ARIMA' in x else
                             'blue'
                             for x in model_counts.index],
                showlegend=False
            ),
            row=3, col=1
        )

        # 4. 신뢰도 분포
        confidence_by_model = pred_df.groupby('모델')['신뢰도'].mean().sort_values(ascending=False)

        fig.add_trace(
            go.Bar(
                x=confidence_by_model.index,
                y=confidence_by_model.values,
                name='평균 신뢰도',
                marker_color='lightcoral',
                text=confidence_by_model.values.round(2),
                textposition='outside',
                showlegend=False
            ),
            row=4, col=1
        )

        # 레이아웃 설정
        fig.update_layout(
            height=1000,
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            title_text=f"고급 AI 시계열 예측 결과 (Prophet/LSTM/ARIMA/ML 앙상블)"
        )

        # Y축 설정
        fig.update_yaxes(title_text="예상 승인수 (건)", row=1, col=1)
        fig.update_yaxes(title_text="전체 승인수 (건)", row=2, col=1)
        fig.update_yaxes(title_text="예측 횟수", row=3, col=1)
        fig.update_yaxes(title_text="신뢰도", row=4, col=1)
        fig.update_xaxes(title_text="날짜", row=2, col=1)

        # 주말 구분선 추가
        for date in date_range:
            if date.dayofweek in [5, 6]:
                for row in [1, 2]:
                    fig.add_vrect(
                        x0=date,
                        x1=date + pd.Timedelta(days=1),
                        fillcolor="LightGray",
                        opacity=0.2,
                        layer="below",
                        line_width=0,
                        row=row, col=1
                    )

        # 평균선 추가
        avg_total = daily_total['예상_승인수'].mean()
        fig.add_hline(
            y=avg_total,
            line_dash="dash",
            line_color="red",
            annotation_text=f"일 평균: {avg_total:.0f}건",
            row=2, col=1
        )

        # 예측 요약 정보
        total_expected = daily_total['예상_승인수'].sum()
        model_usage = ", ".join([f"{model}: {count}회" for model, count in model_counts.items()])
        avg_confidence = pred_df['신뢰도'].mean()

        fig.add_annotation(
            text=f"<b>예측 기간 총 예상 승인수: {total_expected:,.0f}건</b><br>" +
                 f"모델 사용: {model_usage}<br>" +
                 f"평균 예측 신뢰도: {avg_confidence:.1%}",
            xref="paper", yref="paper",
            x=0.5, y=-0.15,
            showarrow=False,
            font=dict(size=14)
        )

        return fig

    except Exception as e:
        print(f"시계열 예측 그래프 생성 오류: {str(e)}")
        return None

def train_prediction_model() -> Tuple[Dict, str]:
    """리드 전환 예측 모델 학습 (시계열 모델 포함)"""
    try:
        if not app_state.is_preprocessed or app_state.processed_data is None:
            return {}, "❌ 먼저 데이터를 전처리해주세요."

        # train_prediction_model_with_data 함수 호출
        return train_prediction_model_with_data(app_state.processed_data)

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return {}, f"❌ 모델 학습 중 오류 발생:\n{str(e)}\n\n상세 오류:\n{error_details}"

def predict_manager_performance(prediction_date: pd.Timestamp = None, prediction_days: int = 30) -> pd.DataFrame:
    """담당자별 예상 승인율 예측 (날짜 기반)"""
    try:
        if not app_state.is_model_trained or app_state.best_model is None:
            return pd.DataFrame()

        if app_state.manager_performance is None or len(app_state.manager_performance) == 0:
            return pd.DataFrame()

        # 예측 날짜 설정
        if prediction_date is None:
            prediction_date = pd.Timestamp.now()

        predictions = []

        # 각 담당자별로 예측
        for _, manager_row in app_state.manager_performance.iterrows():
            manager = manager_row['담당자명']

            if manager == '미배정':
                continue

            # 담당자의 평균 특성 사용
            manager_data = app_state.processed_data[app_state.processed_data['담당자명'] == manager]

            if len(manager_data) == 0:
                continue

            # 예측 기간 동안의 평균 승인율 계산
            daily_predictions = []

            for days_ahead in range(0, prediction_days, 7):  # 주 단위로 샘플링
                future_date = prediction_date + pd.Timedelta(days=days_ahead)

                # 예측 데이터 준비 (더 많은 피처 사용)
                sample_data = manager_data.mean(numeric_only=True).to_dict()

                # 날짜 관련 피처 업데이트
                sample_data.update({
                    '등록_월': future_date.month,
                    '등록_일': future_date.day,
                    '등록_요일': future_date.dayofweek,
                    '등록_시간': 14,
                    '담당자명': manager
                })

                # DataFrame으로 변환
                sample_df = pd.DataFrame([sample_data])

                # 인코딩
                for col, encoder in app_state.encoders.items():
                    if col in sample_df.columns:
                        try:
                            sample_df[f'{col}_encoded'] = encoder.transform(sample_df[col])
                        except:
                            sample_df[f'{col}_encoded'] = 0

                # 피처 선택 및 스케일링
                try:
                    X_pred = sample_df[app_state.feature_names].fillna(0)
                    if app_state.scaler:
                        X_pred = app_state.scaler.transform(X_pred)
                    if app_state.feature_selector:
                        X_pred = app_state.feature_selector.transform(X_pred)

                    pred_proba = app_state.best_model.predict_proba(X_pred)[0, 1]
                    daily_predictions.append(pred_proba)
                except:
                    daily_predictions.append(0.5)

            # 평균 예측 승인율 계산
            avg_pred_proba = np.mean(daily_predictions) if daily_predictions else 0.5

            # 현재 승인건수 계산
            current_approval_count = int(manager_row['승인완료수'])
            total_leads = int(manager_row['총_리드수'])

            # 예측 기간 동안의 예상 리드수 (일일 평균 * 예측 일수)
            daily_avg_leads = total_leads / 30  # 한 달 기준
            expected_leads = int(daily_avg_leads * prediction_days)

            # 예측 승인건수 계산
            predicted_approval_count = int(avg_pred_proba * expected_leads)

            predictions.append({
                '담당자': manager,
                '총리드수': total_leads,
                '평균접촉횟수': round(manager_row['평균_접촉횟수'], 1),
                '현재 승인건': current_approval_count,
                '현재 승인율': f"{manager_row['승인완료율']:.1%}",
                '예측 승인건': predicted_approval_count,
                '예측 승인율': f"{avg_pred_proba:.1%}",
                '예측 정확도': f"{(1 - abs(avg_pred_proba - manager_row['승인완료율'])):.1%}"
            })

        if len(predictions) == 0:
            return pd.DataFrame()

        # 결과 정리
        result_df = pd.DataFrame(predictions)

        # 예측 승인율 기준으로 정렬
        result_df['sort_key'] = result_df['예측 승인율'].str.rstrip('%').astype(float)
        result_df = result_df.sort_values('sort_key', ascending=False).drop('sort_key', axis=1)

        # 예측 기간 정보 추가 (첫 행에만 표시)
        if len(result_df) > 0:
            result_df.loc[result_df.index[0], '예측기간'] = f"{prediction_date.strftime('%Y-%m-%d')} ~ {(prediction_date + pd.Timedelta(days=prediction_days)).strftime('%Y-%m-%d')}"
            # 나머지 행은 빈 문자열
            result_df.loc[result_df.index[1:], '예측기간'] = ''

        return result_df

    except Exception as e:
        print(f"예측 중 오류: {str(e)}")
        return pd.DataFrame()

# ================================================================================
# 11. 리드 배정 최적화 (개선된 버전)
# ================================================================================

def optimize_lead_assignment(channel, gender, type_, memo_len, marketing, email, sms) -> Tuple[str, str, go.Figure]:
    """신규 리드에 대한 최적 담당자 추천 (개선된 버전)"""
    try:
        if not app_state.is_model_trained or app_state.best_model is None:
            return "", "❌ 먼저 모델을 학습시켜주세요.", None

        if app_state.manager_performance is None or len(app_state.manager_performance) == 0:
            return "", "❌ 담당자 데이터가 없습니다.", None

        # 신규 리드 정보 (확장된 피처)
        current_datetime = datetime.now()

        # 기본 피처
        new_lead = {
            '판매처': channel,
            '성별': gender,
            '유형': type_,
            '메모_길이': memo_len,
            '메모_단어수': memo_len // 5,
            '마케팅활용동의_num': int(marketing),
            '이메일수신동의_num': int(email),
            '문자수신동의_num': int(sms),
            '접촉횟수': 0,
            '등록_월': current_datetime.month,
            '등록_일': current_datetime.day,
            '등록_요일': current_datetime.weekday(),
            '등록_시간': current_datetime.hour,
            '동의점수': int(marketing) + int(email) + int(sms),
            '완전동의': int(marketing and email and sms),
            '주말여부': int(current_datetime.weekday() >= 5),
            '월초': int(current_datetime.day <= 10),
            '월말': int(current_datetime.day >= 21),
            '분기': (current_datetime.month - 1) // 3 + 1
        }

        # 메모 품질 점수 계산
        new_lead['메모_품질점수'] = (
            new_lead['메모_길이'] / 500 * 0.3 +  # 최대 500자 기준
            new_lead['메모_단어수'] / 100 * 0.3 +  # 최대 100단어 기준
            0.4  # 키워드 포함 여부 (가정)
        )

        # 고객 프로파일
        new_lead['고객프로파일'] = f"{gender}_{type_}"

        # 각 담당자별 예상 승인확률 계산
        predictions = []

        for _, manager_row in app_state.manager_performance.head(10).iterrows():
            manager = manager_row['담당자명']

            if manager == '미배정':
                continue

            # 예측 데이터 준비
            pred_data = new_lead.copy()
            pred_data['담당자명'] = manager

            # 담당자 관련 피처 추가
            pred_data['담당자_승인율'] = manager_row['승인완료율']
            pred_data['담당자_총승인수'] = manager_row['승인완료수']
            pred_data['담당자_총리드수'] = manager_row['총_리드수']
            pred_data['담당자_평균접촉'] = manager_row['평균_접촉횟수']
            pred_data['담당자_경험지수'] = manager_row['총_리드수'] / app_state.manager_performance['총_리드수'].max()
            pred_data['담당자_효율성'] = manager_row['승인완료율'] / (manager_row['평균_접촉횟수'] + 1)

            # 판매처 관련 피처 (전체 평균 사용)
            channel_stats = app_state.processed_data.groupby('판매처')['is_approved'].mean()
            pred_data['판매처_평균승인율'] = channel_stats.get(channel, channel_stats.mean())

            # 상호작용 피처
            pred_data['담당자X판매처_시너지'] = pred_data['담당자_승인율'] * pred_data['판매처_평균승인율']

            # DataFrame으로 변환
            pred_df = pd.DataFrame([pred_data])

            # 인코딩
            for col, encoder in app_state.encoders.items():
                if col in pred_df.columns:
                    try:
                        pred_df[f'{col}_encoded'] = encoder.transform(pred_df[col])
                    except:
                        pred_df[f'{col}_encoded'] = 0

            # 예측
            try:
                X_pred = pred_df[app_state.feature_names].fillna(0)

                # 스케일링
                if app_state.scaler:
                    X_pred = app_state.scaler.transform(X_pred)

                # 피처 선택
                if app_state.feature_selector:
                    X_pred = app_state.feature_selector.transform(X_pred)

                # 예측
                if app_state.ensemble_model:
                    # 앙상블 모델 우선 사용
                    pred_proba = app_state.ensemble_model.predict_proba(X_pred)[0, 1]
                else:
                    pred_proba = app_state.best_model.predict_proba(X_pred)[0, 1]

            except:
                pred_proba = 0.5

            # 워크로드 고려 (현재 리드수가 많은 담당자는 페널티)
            workload_penalty = 1 - (manager_row['총_리드수'] / app_state.manager_performance['총_리드수'].max()) * 0.1
            adjusted_proba = pred_proba * workload_penalty

            predictions.append({
                '담당자': manager,
                '예상_승인확률': pred_proba,
                '조정_승인확률': adjusted_proba,
                '과거_승인율': manager_row['승인완료율'],
                '현재_리드수': manager_row['총_리드수'],
                '평균_접촉횟수': manager_row['평균_접촉횟수'],
                '담당자_효율성': pred_data['담당자_효율성'],
                '종합점수': adjusted_proba * 0.6 + manager_row['승인완료율'] * 0.3 + pred_data['담당자_효율성'] * 0.1
            })

        if len(predictions) == 0:
            return "", "❌ 예측할 수 있는 담당자가 없습니다.", None

        # 결과 정렬
        pred_df = pd.DataFrame(predictions).sort_values('종합점수', ascending=False)

        # 최적 담당자 선택
        best_manager = pred_df.iloc[0]['담당자']
        best_prob = pred_df.iloc[0]['예상_승인확률']
        best_adjusted_prob = pred_df.iloc[0]['조정_승인확률']

        # 시각화 (개선된 버전)
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                '담당자별 예상 승인확률',
                '종합 점수 (워크로드 고려)',
                '담당자 효율성 지표',
                '추천 근거 분석'
            ),
            specs=[[{"type": "bar"}, {"type": "bar"}],
                   [{"type": "scatter"}, {"type": "bar"}]]
        )

        # 상위 5명만 표시
        top_5 = pred_df.head(5)

        # 1. 예상 승인확률
        fig.add_trace(
            go.Bar(
                x=top_5['담당자'],
                y=top_5['예상_승인확률'],
                name='예상 승인확률',
                marker_color='lightblue',
                text=top_5['예상_승인확률'].apply(lambda x: f'{x:.1%}'),
                textposition='outside'
            ),
            row=1, col=1
        )

        # 과거 승인율 추가
        fig.add_trace(
            go.Bar(
                x=top_5['담당자'],
                y=top_5['과거_승인율'],
                name='과거 평균',
                marker_color='lightgreen',
                text=top_5['과거_승인율'].apply(lambda x: f'{x:.1%}'),
                textposition='outside'
            ),
            row=1, col=1
        )

        # 2. 종합 점수
        fig.add_trace(
            go.Bar(
                x=top_5['담당자'],
                y=top_5['종합점수'],
                name='종합 점수',
                marker_color='orange',
                text=top_5['종합점수'].apply(lambda x: f'{x:.3f}'),
                textposition='outside',
                showlegend=False
            ),
            row=1, col=2
        )

        # 3. 효율성 vs 워크로드
        fig.add_trace(
            go.Scatter(
                x=top_5['현재_리드수'],
                y=top_5['담당자_효율성'],
                mode='markers+text',
                marker=dict(
                    size=top_5['예상_승인확률'] * 50,
                    color=top_5['종합점수'],
                    colorscale='RdYlGn',
                    showscale=True
                ),
                text=top_5['담당자'],
                textposition='top center',
                name='효율성',
                showlegend=False
            ),
            row=2, col=1
        )

        # 4. 추천 근거 요약
        factors = ['예상 승인률', '과거 성과', '효율성', '워크로드']
        best_factors = [
            pred_df.iloc[0]['예상_승인확률'],
            pred_df.iloc[0]['과거_승인율'],
            pred_df.iloc[0]['담당자_효율성'],
            1 - pred_df.iloc[0]['현재_리드수'] / pred_df['현재_리드수'].max()
        ]

        fig.add_trace(
            go.Bar(
                x=factors,
                y=best_factors,
                name='평가 지표',
                marker_color=['blue', 'green', 'orange', 'red'],
                text=[f'{v:.2f}' for v in best_factors],
                textposition='outside',
                showlegend=False
            ),
            row=2, col=2
        )

        # 레이아웃 설정
        fig.update_layout(
            height=800,
            showlegend=True,
            title_text=f'AI 기반 최적 담당자 추천: {best_manager}'
        )

        fig.update_xaxes(title_text="담당자", row=1, col=1)
        fig.update_xaxes(title_text="담당자", row=1, col=2)
        fig.update_xaxes(title_text="현재 리드수", row=2, col=1)
        fig.update_xaxes(title_text="평가 요소", row=2, col=2)

        fig.update_yaxes(title_text="확률", row=1, col=1)
        fig.update_yaxes(title_text="점수", row=1, col=2)
        fig.update_yaxes(title_text="효율성 지수", row=2, col=1)
        fig.update_yaxes(title_text="지표값", row=2, col=2)

        # 추천 이유 생성 (더 상세한 버전)
        recommendation_text = f"""🎯 최적 담당자 추천: {best_manager}

📊 추천 근거:
- 예상 승인 확률: {best_prob:.1%} (조정 후: {best_adjusted_prob:.1%})
- 과거 평균 승인율: {pred_df.iloc[0]['과거_승인율']:.1%}
- 현재 처리 중인 리드: {int(pred_df.iloc[0]['현재_리드수'])}건
- 평균 접촉 횟수: {pred_df.iloc[0]['평균_접촉횟수']:.1f}회
- 담당자 효율성: {pred_df.iloc[0]['담당자_효율성']:.3f}

💡 추천 이유:
1. 이 담당자는 유사한 프로파일의 리드에서 높은 성과를 보였습니다.
2. 현재 워크로드가 적절한 수준으로 추가 리드 처리가 가능합니다.
3. 효율성 지표가 우수하여 적은 접촉으로도 높은 전환율을 달성합니다.
4. AI 모델이 이 리드의 특성과 담당자의 강점이 매치된다고 예측합니다.

📈 대안 담당자:
"""

        for i in range(1, min(3, len(pred_df))):
            alt = pred_df.iloc[i]
            recommendation_text += f"\n{i+1}. {alt['담당자']} (예상: {alt['예상_승인확률']:.1%}, 종합: {alt['종합점수']:.3f})"

        # 리드 특성 요약
        recommendation_text += f"""

📋 리드 특성 요약:
- 판매처: {channel}
- 고객 프로파일: {gender} / {type_}
- 동의 점수: {new_lead['동의점수']}/3
- 예상 메모 품질: {new_lead['메모_품질점수']:.2f}
- 시간대 특성: {'주말' if new_lead['주말여부'] else '평일'}, {current_datetime.hour}시
"""

        return best_manager, recommendation_text, fig

    except Exception as e:
        return "", f"❌ 예측 중 오류 발생: {str(e)}", None

# ================================================================================
# 12. Gradio 인터페이스 구성 (기존 유지)
# ================================================================================

def create_gradio_interface():
    """Gradio 인터페이스 생성"""

    # CSS 스타일
    custom_css = """
    .gradio-container {
        font-family: 'Noto Sans KR', -apple-system, BlinkMacSystemFont, sans-serif !important;
    }
    .gr-button-primary {
        background-color: #2563eb !important;
        color: white !important;
    }
    .gr-button-primary:hover {
        background-color: #1d4ed8 !important;
    }
    .error-text {
        color: #dc2626 !important;
        font-weight: bold;
    }
    .success-text {
        color: #16a34a !important;
        font-weight: bold;
    }
    """

    with gr.Blocks(title="담당자별 리드 전환 예측 시스템", theme=gr.themes.Soft(), css=custom_css) as app:

        # 헤더
        gr.Markdown("""
        # 🎯 담당자별 리드 전환 예측 시스템

        ### 📊 AI 기반 리드 배정 최적화 및 성과 예측 플랫폼

        ---
        """)

        # 메인 탭
        with gr.Tabs():

            # 1. 데이터 업로드 및 전처리
            with gr.Tab("📁 데이터 관리"):
                with gr.Row():
                    with gr.Column(scale=1):
                        file_input = gr.File(
                            label="엑셀 파일 업로드 (tst_atalk.xlsx)",
                            file_types=[".xlsx", ".xls", ".csv"],
                            type="filepath"
                        )

                        with gr.Row():
                            upload_btn = gr.Button("📤 데이터 로드", variant="primary")
                            sample_btn = gr.Button("🎲 샘플 데이터 사용", variant="secondary")

                        preprocess_btn = gr.Button("🔧 전처리 실행", variant="primary", size="lg")

                    with gr.Column(scale=2):
                        data_info = gr.Textbox(
                            label="데이터 분석 결과",
                            lines=20,
                            interactive=False
                        )

                # 이벤트 핸들러
                upload_btn.click(
                    fn=load_data,
                    inputs=[file_input],
                    outputs=[gr.State(), data_info]
                )

                sample_btn.click(
                    fn=lambda: load_data(None),
                    outputs=[gr.State(), data_info]
                )

                preprocess_btn.click(
                    fn=preprocess_for_prediction,
                    outputs=[gr.State(), data_info]
                )

            # 2. 담당자 성과 분석
            with gr.Tab("👥 담당자 성과 분석"):
                analyze_btn = gr.Button("📊 성과 분석 실행", variant="primary", size="lg")

                status_text = gr.Textbox(label="상태", interactive=False, visible=True)

                with gr.Row():
                    perf_plot1 = gr.Plot(label="성과 매트릭스")
                    perf_plot2 = gr.Plot(label="승인율 순위")

                with gr.Row():
                    perf_plot3 = gr.Plot(label="일별 추이")
                    perf_plot4 = gr.Plot(label="접촉 효율성")

                def analyze_performance():
                    plots, message = create_manager_performance_dashboard()
                    if len(plots) >= 4:
                        return message, plots[0], plots[1], plots[2], plots[3]
                    elif len(plots) == 3:
                        return message, plots[0], plots[1], plots[2], None
                    elif len(plots) == 2:
                        return message, plots[0], plots[1], None, None
                    elif len(plots) == 1:
                        return message, plots[0], None, None, None
                    else:
                        return message, None, None, None, None

                analyze_btn.click(
                    fn=analyze_performance,
                    outputs=[status_text, perf_plot1, perf_plot2, perf_plot3, perf_plot4]
                )

            # 3. 예측 모델 학습
            with gr.Tab("🤖 AI 모델 학습"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("### 📅 학습 기간 설정")

                        # 날짜 선택 컴포넌트
                        with gr.Row():
                            train_start_date = gr.Textbox(
                                label="학습 시작일",
                                value="2024-01-01",
                                info="YYYY-MM-DD 형식"
                            )
                            train_end_date = gr.Textbox(
                                label="학습 종료일",
                                value=datetime.now().strftime("%Y-%m-%d"),
                                info="YYYY-MM-DD 형식"
                            )

                        gr.Markdown("### 🔮 예측 기간 설정")

                        with gr.Row():
                            predict_start_date = gr.Textbox(
                                label="예측 시작일",
                                value=datetime.now().strftime("%Y-%m-%d"),
                                info="YYYY-MM-DD 형식"
                            )
                            predict_days = gr.Slider(
                                label="예측 일수",
                                minimum=1,
                                maximum=90,
                                value=30,
                                step=1,
                                info="며칠 후까지 예측할지 설정"
                            )

                        train_btn = gr.Button("🚀 모델 학습 및 예측 시작", variant="primary", size="lg")

                        model_info = gr.Textbox(
                            label="모델 학습 결과",
                            lines=15,
                            interactive=False
                        )

                    with gr.Column(scale=2):
                        with gr.Tabs():
                            with gr.Tab("📈 담당자별 예상 성과"):
                                prediction_table = gr.Dataframe(
                                    label="담당자별 예측 결과",
                                    interactive=False
                                )

                            with gr.Tab("📊 시계열 예측"):
                                time_series_plot = gr.Plot(
                                    label="기간별 예측 추이"
                                )

                def train_and_predict_with_dates(start_date, end_date, pred_start, pred_days):
                    try:
                        # 날짜 형식 검증
                        start = pd.to_datetime(start_date)
                        end = pd.to_datetime(end_date)
                        pred_start_dt = pd.to_datetime(pred_start)

                        if start > end:
                            return "❌ 시작일이 종료일보다 늦습니다.", pd.DataFrame(), None

                        # 지정된 기간의 데이터만 필터링
                        filtered_data = filter_data_by_date(start, end)

                        if filtered_data is None or len(filtered_data) == 0:
                            return "❌ 선택한 기간에 데이터가 없습니다.", pd.DataFrame(), None

                        # 모델 학습
                        results, text = train_prediction_model_with_data(filtered_data)

                        if app_state.is_model_trained:
                            # 지정된 날짜 기준으로 성과 예측
                            pred_df = predict_manager_performance(pred_start_dt, pred_days)

                            # 시계열 예측
                            time_series_fig = create_time_series_prediction(pred_start_dt, pred_days)

                            # 학습 정보 추가
                            text += f"\n\n📅 학습 기간: {start.date()} ~ {end.date()}"
                            text += f"\n🔮 예측 기간: {pred_start_dt.date()} ~ {(pred_start_dt + pd.Timedelta(days=pred_days)).date()}"
                            text += f"\n📊 예측 일수: {pred_days}일"

                            return text, pred_df, time_series_fig

                        return text, pd.DataFrame(), None

                    except Exception as e:
                        return f"❌ 오류 발생: {str(e)}", pd.DataFrame(), None

                train_btn.click(
                    fn=train_and_predict_with_dates,
                    inputs=[train_start_date, train_end_date, predict_start_date, predict_days],
                    outputs=[model_info, prediction_table, time_series_plot]
                )

            # 4. 리드 배정 최적화
            with gr.Tab("🎯 리드 배정 최적화"):
                gr.Markdown("### 신규 리드 정보 입력")

                with gr.Row():
                    with gr.Column():
                        channel_input = gr.Dropdown(
                            label="판매처",
                            choices=['G마켓', 'CJ몰', '11번가', 'SSG', '옥션', '네이버', '쿠팡', '기타'],
                            value="G마켓"
                        )
                        gender_input = gr.Radio(
                            label="성별",
                            choices=["남자", "여자"],
                            value="여자"
                        )
                        type_input = gr.Radio(
                            label="유형",
                            choices=["개인", "법인"],
                            value="개인"
                        )
                        memo_length = gr.Slider(
                            label="메모 길이",
                            minimum=0,
                            maximum=500,
                            value=100,
                            step=10
                        )
                        marketing_consent = gr.Checkbox(label="마케팅 동의", value=True)
                        email_consent = gr.Checkbox(label="이메일 수신 동의", value=True)
                        sms_consent = gr.Checkbox(label="문자 수신 동의", value=True)

                        assign_btn = gr.Button("🎯 최적 담당자 찾기", variant="primary", size="lg")

                    with gr.Column():
                        best_manager = gr.Textbox(
                            label="추천 담당자",
                            interactive=False
                        )
                        recommendation = gr.Textbox(
                            label="추천 상세 정보",
                            lines=15,
                            interactive=False
                        )
                        comparison_plot = gr.Plot(label="담당자별 예상 성과")

                assign_btn.click(
                    fn=optimize_lead_assignment,
                    inputs=[channel_input, gender_input, type_input, memo_length,
                           marketing_consent, email_consent, sms_consent],
                    outputs=[best_manager, recommendation, comparison_plot]
                )

        # 푸터
        gr.Markdown("""
        ---

        ### 💡 사용 가이드

        1. **데이터 업로드**: tst_atalk.xlsx 파일을 업로드하거나 샘플 데이터를 사용합니다.
        2. **전처리 실행**: 데이터를 분석에 적합한 형태로 변환합니다.
        3. **성과 분석**: 담당자별 현재 성과를 다각도로 분석합니다.
        4. **모델 학습**: AI 모델을 학습시켜 담당자별 예상 성과를 예측합니다.
        5. **리드 배정**: 신규 리드에 대해 최적의 담당자를 추천받습니다.

        ---

        © 2024 Lead Conversion Prediction System. All rights reserved.
        """)

    return app

# ================================================================================
# 13. 메인 실행
# ================================================================================

if __name__ == "__main__":
    print("🚀 담당자별 리드 전환 예측 시스템 시작 중...")
    print("📌 샘플 데이터로 빠르게 시작하려면 [샘플 데이터 사용] 버튼을 클릭하세요!")

    # 사용 가능한 모델 확인
    print("\n📊 사용 가능한 모델:")
    print(f"- 기본 ML 모델: ✅ (Random Forest, XGBoost, LightGBM 등)")
    print(f"- LSTM (시계열): {'✅' if TENSORFLOW_AVAILABLE else '❌ (TensorFlow 필요)'}")
    print(f"- ARIMA (시계열): {'✅' if STATSMODELS_AVAILABLE else '❌ (statsmodels/pmdarima 필요)'}")
    print(f"- Prophet (시계열): {'✅' if PROPHET_AVAILABLE else '❌ (prophet 필요)'}")
    print(f"- CatBoost: {'✅' if CATBOOST_AVAILABLE else '❌ (catboost 필요)'}")
    print(f"- 하이퍼파라미터 최적화: {'✅' if OPTUNA_AVAILABLE else '❌ (optuna 필요)'}")
    print(f"- 클래스 불균형 처리: {'✅' if IMBALANCED_AVAILABLE else '❌ (imbalanced-learn 필요)'}")

    print("\n💡 시계열 모델이 없어도 기본 ML 모델만으로 충분한 성능을 얻을 수 있습니다.")

    # Gradio 앱 생성 및 실행
    app = create_gradio_interface()

    # 실행
    app.launch(
        share=True,  # 공유 링크 생성
        debug=False,
        quiet=False
    )

    print("✅ 시스템이 성공적으로 시작되었습니다!")

📦 필요한 패키지 설치 중...
✅ gradio>=5.0.0 설치 완료
✅ pandas 설치 완료
✅ numpy 설치 완료
✅ scikit-learn>=1.0.0 설치 완료
✅ lightgbm 설치 완료
✅ xgboost 설치 완료
✅ catboost 설치 완료
✅ optuna>=3.0.0 설치 완료
✅ plotly 설치 완료
✅ seaborn 설치 완료
✅ matplotlib 설치 완료
✅ openpyxl 설치 완료
✅ japanize-matplotlib 설치 완료
✅ tensorflow 설치 완료
✅ imbalanced-learn>=0.9.0 설치 완료
✅ feature-engine 설치 완료

📈 시계열 분석 패키지 설치 중...
✅ statsmodels 설치 완료
✅ pmdarima 설치 완료
✅ prophet 설치 완료

🎨 한글 폰트 설치 중...

✅ 패키지 설치 완료!
💡 일부 시계열 패키지가 설치되지 않아도 기본 ML 모델은 정상 작동합니다.
✅ TensorFlow 로드 성공
⚠️ Statsmodels 로드 중 예상치 못한 오류: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
✅ Prophet 로드 성공
🚀 담당자별 리드 전환 예측 시스템 시작 중...
📌 샘플 데이터로 빠르게 시작하려면 [샘플 데이터 사용] 버튼을 클릭하세요!

📊 사용 가능한 모델:
- 기본 ML 모델: ✅ (Random Forest, XGBoost, LightGBM 등)
- LSTM (시계열): ✅
- ARIMA (시계열): ❌ (statsmodels/pmdarima 필요)
- Prophet (시계열): ✅
- CatBoost: ✅
- 하이퍼파라미터 최적화: ✅
- 클래스 불균형 처리: ✅

💡 시계열 모델이 없어도 기본 ML 모델만으로 충분한 성능을 얻을 수 있습니다.
Colab notebook detected. To s

✅ 시스템이 성공적으로 시작되었습니다!
